In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer, SimpleImputer
from importlib import reload
import copy

import utils
reload(utils)

<module 'utils' from '/home/tpjoe/tpjoe@stanford.edu/project_NACC/scripts/utils.py'>

In [2]:
# Load dataframe, rename one column (to make it conform with description dataframe) and sort by patients ID
# df = pd.read_csv('../data/phongpreecha03152021.csv', low_memory=False)
df = pd.read_csv('df_samples.csv', low_memory=False)
df = df.rename({'ADCNAME': 'NACCADC'}, axis=1)
df = df.sort_values(['NACCID', 'NACCVNUM'])

# Inspect columns with NAs
colNA = df.isna().sum() > 0
colNA = [colNA.keys().tolist()[i] for i in np.array(np.where(colNA.tolist()))[0, :]]
colNA = pd.DataFrame(df.isna().sum()[colNA]).transpose()
# colNA.iloc[0, :].value_counts().index.to_list()

In [3]:
# Get only UDS, this would eradicate the 8000NA peak
df_UDS = df.loc[df.NACCMDSS != 2, :] # most features in NACCMDSS == 2 are just NP and only MMSE available, no lifestyle data
# df_UDS = df_UDS.loc[:, (df_UDS.isna().sum() < 500)]

# creating visit feature 
visit = df_UDS.groupby(['NACCID']).cumcount() + 1

# read back those marked features
marked_features = pd.read_csv('NA_less_than_20000_new_type_filled.csv')
df_UDS = df_UDS.loc[:, df_UDS.columns.isin(marked_features.var_name.values)]

# split data to X and y
# y_col_UDS = pd.DataFrame(df_UDS.columns)[0].apply(lambda i: True if (marked_features.loc[marked_features.var_name==i, 'type'].values[0]=='y') else False).tolist()
x_col_UDS = pd.DataFrame(df_UDS.columns)[0].apply(lambda i: True if (marked_features.loc[marked_features.var_name==i, 'type'].values[0]=='x') else False).tolist()
X_UDS = df_UDS.loc[:, ['NACCID', 'NACCVNUM'] + list(df_UDS.columns[x_col_UDS])]
# y_UDS = df_UDS.loc[:, ['NACCID', 'NACCVNUM'] + list(df_UDS.columns[y_col_UDS])]

# Let's select just those that have meaningful amount of data first
# y_UDS = y_UDS.loc[:, y_UDS.applymap(lambda x: (x==88.8)|(x==-4)).sum() < 10000]
X_UDS = X_UDS.loc[:, X_UDS.loc[df.NACCVNUM==1, :].applymap(lambda x: x==-4).sum() < 1000]
# Xy_UDS = pd.concat([X_UDS, y_UDS], axis=1, sort=False)

In [4]:
# start preprocessing
guidelines = pd.read_csv('processing_guidelines_updated.csv')

# start with those that will be dropped (D)
drop_features = guidelines.loc[['D' in c for c in guidelines.cleared.tolist()], 'feature'].tolist()
if any(i in X_UDS.columns for i in drop_features):
    X_UDS = X_UDS.drop(drop_features, axis=1)

# PRIMLANG
X_UDS.loc[X_UDS.PRIMLANG.isin([3, 4, 6, 9]), 'PRIMLANG'] = 8 # get rid of some minor lagnaguses

# those where -4 and 9 must be combined (c)
combine_features = guidelines.loc[[c=='c' for c in guidelines.cleared.tolist()], 'feature'].tolist()
combine_features = [i for i in combine_features if i != 'PRIMLANG']
X_UDS[combine_features] = X_UDS[combine_features].stack().replace({-4:9}).unstack()

# those that -4 needs to be replaced by copied value from the 1st visit
dup_features = ['NPSYLAN']
X_UDS.loc[:, dup_features] = X_UDS.loc[:, dup_features].replace(-4, np.nan) # no first visit = -4
X_UDS.loc[:, dup_features] = X_UDS.loc[:, dup_features].fillna(method='ffill')

# impute only (i; continuos)
impCon_features = guidelines.loc[(guidelines.cleared=='i') & (guidelines['categ']=='n'), 'feature'].tolist()
for i, col in enumerate(impCon_features):
    X_UDS.loc[:, col] = X_UDS.loc[:, col].replace([-4, 99], np.nan)
    X_UDS.loc[:, col] = X_UDS.loc[:, col].fillna(X_UDS.loc[:, col].mean())

# impute only (i; categ)
impCateg_features = guidelines.loc[(guidelines.cleared=='i') & (guidelines['categ']=='y'), 'feature'].tolist()
for i, col in enumerate(impCateg_features):
    X_UDS.loc[:, col] = X_UDS.loc[:, col].replace([9, 99], X_UDS.loc[:, col].value_counts().idxmax())

# those needing imputation and duplicates (ic)
ic_features = guidelines.loc[[c=='ic' for c in guidelines.cleared.tolist()], 'feature'].tolist()
ic_features = [i for i in ic_features if i != 'DECAGE']
for i, col in enumerate(ic_features):
    X_UDS.loc[:, col] = X_UDS.loc[:, col].replace([-4, 9], np.nan)
    X_UDS.loc[:, col] = X_UDS.loc[:, col].fillna(X_UDS.loc[:, col].mean())

X_UDS[ic_features] = X_UDS[ic_features].stack().replace({8:0}).unstack()

# special ic: DECAGE
X_UDS.loc[:, 'DECAGE'] = X_UDS.loc[:, 'DECAGE'].replace([999], np.nan)
X_UDS.loc[:, 'DECAGE'] = X_UDS.loc[:, 'DECAGE'].fillna(X_UDS.loc[:, 'DECAGE'].mean())
X_UDS.loc[X_UDS.DECAGE!=888, 'DECAGE'] = 1/X_UDS.loc[X_UDS.DECAGE!=888, 'DECAGE']
X_UDS.loc[X_UDS.DECAGE==888, 'DECAGE'] = 0

# those needing imputation and duplicates (id)
id_features = guidelines.loc[['id' == c for c in guidelines.cleared.tolist()], 'feature'].tolist()
X_UDS.loc[:, id_features] = X_UDS.loc[:, id_features].replace(9, 0) # most are 0s in the first visit
X_UDS.loc[:, id_features] = X_UDS.loc[:, id_features].replace(-4, np.nan)
X_UDS[id_features] = X_UDS.groupby('NACCID')[id_features].apply(lambda x: x.fillna(method='ffill'))

# those needing imputation and duplicates (icd non-categ)
X_UDS.loc[(df_UDS.NACCVNUM==1) & ((X_UDS.SMOKYRS==88) | (X_UDS.SMOKYRS==99)), 'SMOKYRS'] = np.nan
X_UDS['SMOKYRS'] = X_UDS['SMOKYRS'].fillna(X_UDS['SMOKYRS'].mean())
year_inc = pd.concat([X_UDS[['NACCID', 'NACCAGE']].groupby(['NACCID']).diff(), X_UDS.NACCID], axis=1).groupby('NACCID').cumsum().fillna(0)
smok_1stvisit = X_UDS.loc[df_UDS.NACCVNUM==1, 'SMOKYRS'].repeat(X_UDS.NACCID.value_counts())
smok_1stvisit.index = year_inc.index
year_smok = year_inc.NACCAGE + smok_1stvisit
X_UDS.loc[X_UDS['SMOKYRS']==-4, 'SMOKYRS'] = year_smok.loc[X_UDS.SMOKYRS==-4]

# icd, inverse age, and categ
icd_noncateg = guidelines.loc[(guidelines.cleared == 'icd') & (guidelines.categ == 'n'), 'feature'][1:].values.tolist()
year_inc = pd.concat([X_UDS[['NACCID', 'NACCAGE']].groupby(['NACCID']).diff(), X_UDS.NACCID], axis=1).groupby('NACCID').cumsum().fillna(0)
for col in icd_noncateg:
    X_UDS.loc[X_UDS[col] == -4, col] = np.nan
    X_UDS.loc[:, col] = X_UDS.loc[:, col].fillna(method='ffill')
    if col == 'QUITSMOK':
        X_UDS.loc[X_UDS[col] == 888, col] = 0
        X_UDS.loc[X_UDS[col] == 999, col] = X_UDS.loc[X_UDS[col] != 999, col].mean()
    else:
        X_UDS.loc[~X_UDS[col].isin([8888, 9999]), col] = 1/X_UDS.loc[~X_UDS[col].isin([8888, 9999]), col]
        X_UDS.loc[X_UDS[col] == 8888, col] = 0
        X_UDS.loc[X_UDS[col] == 9999, col] = X_UDS.loc[X_UDS[col] != 9999, col].mean()

/tmp/ipykernel_83266/2846450979.py:52: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  X_UDS[id_features] = X_UDS.groupby('NACCID')[id_features].apply(lambda x: x.fillna(method='ffill'))


In [5]:
# r features
replace_dict = {'SPEECH': {'replacedBy': 'APRAXSP', 'nonzerokey':{1:2}},
                'TRESTRHD': {'replacedBy': 'RESTTRL', 'nonzerokey':{1:2}},
                'TRESTLHD': {'replacedBy': 'RESTTRR', 'nonzerokey':{1:2}},
                'TRACTRHD': {'replacedBy': 'POSTINST', 'nonzerokey':{1:2}},
                'TRACTLHD': {'replacedBy': 'POSTINST', 'nonzerokey':{1:2}},
                'LEGRT': {'replacedBy': 'CVDMOTR', 'nonzerokey':{1:2}},
                'LEGLF': {'replacedBy': 'CVDMOTL', 'nonzerokey':{1:2}},
                'POSTURE': {'replacedBy': 'POSTINST', 'nonzerokey':{1:2}},
                'POSSTAB': {'replacedBy': 'POSTINST', 'nonzerokey':{1:2}},
                'BRADYKIN': {'replacedBy': 'BRADY', 'nonzerokey':{1:1}},
                'NACCPPME': {'replacedBy': 'NACCPPAG', 'nonzerokey':{1:2, 2:4, 3:1, 4:4, 7:7, 8:8, -4:8}},
                'VASC': {'replacedBy': 'CVD', 'nonzerokey':{1:1}},
                'VASCIF': {'replacedBy': 'CVDIF', 'nonzerokey':{1:1, 2:2, 3:7, 7:7, 8:8, 3:7, -4:8}},
                'STROKE': {'replacedBy': 'PREVSTK', 'nonzerokey':{1:1}},
                'STROKIF': {'replacedBy': 'STROKDEC', 'nonzerokey':{1:2, 8:8}}         
                }
r_features = list(replace_dict.keys())
mod_df = df.copy()
if not all(X_UDS.index == mod_df.index): print('SHITTTTTTTTTTTT')

for replaced_f in replace_dict.keys():
    mod_df[replace_dict[replaced_f]['replacedBy']] = mod_df[replace_dict[replaced_f]['replacedBy']].apply(lambda x: 0 \
        if x not in replace_dict[replaced_f]['nonzerokey'].keys() else replace_dict[replaced_f]['nonzerokey'][x])
    replaced_row = X_UDS.loc[X_UDS.loc[:, replaced_f]==-4, :].index    
    X_UDS.loc[replaced_row, replaced_f] = mod_df.loc[replaced_row, replace_dict[replaced_f]['replacedBy']]


# r2 features
r2_features = ['FOCLSIGN', 'FACEXP', 'GAIT']

replaced_f = 'FOCLSIGN'
value_holder = df.loc[:, ['CVDSIGNS', 'SIVDFIND']].apply(lambda x: 2 if ((x.CVDSIGNS==1) & (x.SIVDFIND==1)) else 0, axis=1)
replaced_row = X_UDS.loc[X_UDS.loc[:, replaced_f]==-4, :].index
X_UDS.loc[replaced_row, replaced_f] = value_holder.loc[replaced_row]

replaced_f = 'FACEXP'
value_holder = df.loc[:, ['CVDMOTL', 'CVDMOTR']].apply(lambda x: 2 if ((x.CVDMOTL==1) | (x.CVDMOTR==1)) else 0, axis=1)
replaced_row = X_UDS.loc[X_UDS.loc[:, replaced_f]==-4, :].index
X_UDS.loc[replaced_row, replaced_f] = value_holder.loc[replaced_row]

replaced_f = 'GAIT'
value_holder = df.loc[:, ['PARKGAIT', 'GAITNPH', 'GAITPSP']].apply(lambda x: 2 if ((x.PARKGAIT==1) | (x.GAITNPH==1) | (x.GAITPSP==1)) else 0, axis=1)
replaced_row = X_UDS.loc[X_UDS.loc[:, replaced_f]==-4, :].index
X_UDS.loc[replaced_row, replaced_f] = value_holder.loc[replaced_row]


# ir features
replace_dict = {'CVHATT': {'replacedBy': 'MYOINF', 'nonzerokey':{1:1}},
                'CVAFIB': {'replacedBy': 'AFIBRILL', 'nonzerokey':{1:1}},
                'CVPACE': {'replacedBy': 'PACEMAKE', 'nonzerokey':{1:1}},
                'CVCHF': {'replacedBy': 'CONGHRT', 'nonzerokey':{1:1}},
                'CBSTROKE': {'replacedBy': 'PREVSTK', 'nonzerokey':{1:1}},
                'NACCTBI': {'replacedBy': 'BRNINJ', 'nonzerokey':{1:1}},
                'TRAUMCHR': {'replacedBy': 'BRNINJ', 'nonzerokey':{1:1}},
                'NCOTHR': {'replacedBy': 'OTHCOG', 'nonzerokey':{1:1}},
                'DIABETES': {'replacedBy': 'DIABET', 'nonzerokey':{1:1, 2:1, 3:1}},
                'HYPERTEN': {'replacedBy': 'HYPERT', 'nonzerokey':{1:1}},
                'B12DEF': {'replacedBy': 'VB12DEF', 'nonzerokey':{1:1}},
                'THYROID': {'replacedBy': 'THYDIS', 'nonzerokey':{1:1}},
                'INCONTU': {'replacedBy': 'BOWLINC', 'nonzerokey':{1:1}},
                'INCONTF': {'replacedBy': 'NACCPPAG', 'nonzerokey':{1:1}},
                'ALCOHOL': {'replacedBy': 'ALCABUSE', 'nonzerokey':{1:1}},
                'ABUSOTHR': {'replacedBy': 'IMPSUB', 'nonzerokey':{1:1}},
                'DEP2YRS': {'replacedBy': 'DEPD', 'nonzerokey':{1:1}},
                'DEPOTHR': {'replacedBy': 'DEPD', 'nonzerokey':{1:1}}    
                }
ir_features = list(replace_dict.keys()) + ['CVANGIO', 'TRAUMBRF', 'TRAUMEXT']
mod_df = df.copy()
if not all(X_UDS.index == mod_df.index): print('SHITTTTTTTTTTTT')

for replaced_f in replace_dict.keys():
    mod_df[replace_dict[replaced_f]['replacedBy']] = mod_df[replace_dict[replaced_f]['replacedBy']].apply(lambda x: 0 \
        if x not in replace_dict[replaced_f]['nonzerokey'].keys() else replace_dict[replaced_f]['nonzerokey'][x])
    replaced_row = X_UDS.loc[X_UDS.loc[:, replaced_f]==-4, :].index    
    X_UDS.loc[replaced_row, replaced_f] = mod_df.loc[replaced_row, replace_dict[replaced_f]['replacedBy']]

# special ir cases
value_holder = df.loc[:, ['ANGIOCP', 'ANGIOPCI']].apply(lambda x: 1 if ((x.ANGIOCP==1) | (x.ANGIOPCI==1)) else 0, axis=1)
replaced_row = X_UDS.loc[X_UDS.loc[:, 'CVANGIO']==-4, :].index
X_UDS.loc[replaced_row, 'CVANGIO'] = value_holder.loc[replaced_row]
X_UDS.loc[X_UDS.TRAUMBRF==-4, 'TRAUMBRF'] = df.loc[X_UDS.TRAUMBRF==-4, 'TBIBRIEF']
X_UDS.loc[X_UDS.TRAUMBRF==-4, 'TRAUMBRF'] = df.loc[X_UDS.TRAUMBRF==-4, 'BRNINJ']
X_UDS.loc[X_UDS.TRAUMEXT==-4, 'TRAUMEXT'] = df.loc[X_UDS.TRAUMEXT==-4, 'TBIBRIEF']
X_UDS.loc[X_UDS.TRAUMEXT==-4, 'TRAUMEXT'] = df.loc[X_UDS.TRAUMEXT==-4, 'BRNINJ']

# imputing 9 
X_UDS.loc[:, ir_features] = X_UDS.loc[:, ir_features].replace({9:np.nan})
X_UDS.loc[:, ir_features] = X_UDS.loc[:, ir_features].fillna(X_UDS.loc[:, ir_features].mode().iloc[0])


# s2 cases
s2_features = guidelines.loc[(guidelines.cleared == 's2'), 'feature'].values.tolist()
# first where -4 and 9 must be combined (c) in the first visit
X_UDS.loc[df.NACCVNUM==1, combine_features] = X_UDS.loc[df.NACCVNUM==1, s2_features].stack().replace({-4:9}).unstack()
# then copy from first visit
X_UDS.loc[:, s2_features] = X_UDS.loc[:, s2_features].replace(-4, np.nan) # no first visit = -4
X_UDS.loc[:, s2_features] = X_UDS.loc[:, s2_features].fillna(method='ffill')


# drugs
# reshape druglist to drug columns with 0 and 1 instead
drug_list = df.loc[:, ['DRUG' in j for j in df.columns]].values
drug_list = [pd.DataFrame(np.sort(list(set([i for i in j if str(i)!='nan'])))) for j in drug_list]
for i in drug_list: i.index = i[0].values

drug_df = pd.concat(drug_list, axis=1, sort=False)
drug_df = drug_df.transpose()
drug_df.index = df.index
drug_df = drug_df.fillna(0)
# convert any text to number 1
drug_df = drug_df.apply(pd.to_numeric, errors='coerce')
drug_df = drug_df.fillna(1)
drug_df = drug_df.loc[:, drug_df.sum()>0.01*df.shape[0]] #get drugs with at least ~256 occurences

# there's some NA's left (all categorical)
na_feat = [X_UDS[col].isna().any() for col in X_UDS.columns]
X_UDS.loc[:, na_feat] = X_UDS.loc[:, na_feat].fillna(X_UDS.loc[:, na_feat].mode().iloc[0])

# merge
X_UDS = pd.concat([X_UDS, drug_df], axis=1) 

# those special cases
special_feat = guidelines.loc[guidelines.cleared == 's', 'feature'].values.tolist()
special_feat1 = [i for i in special_feat if i not in ['TRAILA', 'TRAILB', 'WAIS', 'NACCAGE']]
special_feat2 = ['WAIS']
special_feat3 = [i for i in special_feat if i not in special_feat1+special_feat2]

## lump all unknown and use mean of that age and cognitive status for impute
X_UDS_caseII = X_UDS.copy()
X_UDS_caseII.loc[:, special_feat1] = X_UDS_caseII.loc[:, special_feat1].replace({i:np.nan for i in [-4, 88, 95, 96, 97, 98, 99, 995, 996, 997, 998]})
X_UDS_caseII.loc[:, special_feat2] = X_UDS_caseII.loc[:, special_feat2].replace({i:np.nan for i in [-4, 95, 96, 97, 98, 99, 995, 996, 997, 998]})
X_UDS_caseII.loc[:, special_feat3] = X_UDS_caseII.loc[:, special_feat3].replace({i:np.nan for i in [-4, 995, 996, 997, 998]})
imp = KNNImputer(n_neighbors=5)
impute_df = pd.concat([X_UDS_caseII.loc[:, special_feat], \
             X_UDS_caseII.NACCAGE, X_UDS_caseII.NACCUDSD, X_UDS_caseII.EDUC], axis=1)
imputed_df = pd.DataFrame(imp.fit_transform(impute_df), columns=impute_df.columns, index=impute_df.index)
X_UDS_caseII.loc[:, special_feat] = imputed_df.loc[:, special_feat]

# # Impute other scores
other_scores = [i for i in guidelines.loc[(guidelines.cleared=='y') & (guidelines.categ=='n'), 'feature'].values 
                if i not in ['NACCID', 'NACCDAGE', 'NACCAGE', 'NACCAGEB', 'NACCUDSD', 'NACCINT']]
X_UDS_caseII.loc[:, 'EDUC'] = X_UDS_caseII.loc[:, 'EDUC'].replace({99:np.nan}).fillna(X_UDS_caseII.loc[:, 'EDUC'].median()) #99 for EDUC
excluded_columns = np.array(other_scores)[np.where(((X_UDS_caseII.loc[:, other_scores]==-4).sum() > 0).tolist())].tolist()
X_UDS_caseII = X_UDS_caseII.loc[:, ~X_UDS_caseII.columns.isin(excluded_columns)]

# Cases processed
processed_features =  combine_features + dup_features + impCon_features + impCateg_features + id_features + ic_features + \
                     icd_noncateg + r_features + r2_features + ir_features + s2_features + special_feat + other_scores +\
                    ['SMOKYRS', 'PRIMLANG', 'DECAGE', 'TOBAC30', 'TOBAC100'] #+ #drop_features
to_be_processed = [i for i in guidelines.feature[guidelines.cleared!='y'] if i not in processed_features]

/tmp/ipykernel_83266/1697332255.py:138: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_UDS_caseII.loc[:, special_feat] = imputed_df.loc[:, special_feat]


In [17]:
X_UDS_caseII.to_csv('../project_CR/data/X_UDS_caseII_allFeatures.csv')
# y_UDS.to_csv('../project_CR/data/y_UDS_allFeatures.csv')